Copyright **`(c)`** 2023 Zafonte Francesca `<s319331@studenti.polito.it>`  
[`https://github.com/Zafonte/computational-intelligence`](https://github.com/Zafonte/computational-intelligence)  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)



# Tic-Tac-Toe game

In [145]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy
from tqdm.auto import tqdm

import numpy as np
import copy

## Setting of the game

In [146]:
BOARD_ROWS = 3
BOARD_COLS = 3

In [147]:
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.isEnd = False
        self.p1 = p1
        self.p2 = p2
        #init p1 plays first, player x. #-1 is player o
        self.playerSymbol = 1
    

    def get_board(self):
        return  tuple(map(tuple, self.board))
                
     
    def print_Board(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')
    

    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))
        return positions
    

    #board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.isEnd = False
        self.playerSymbol = 1

    
    #CHECK WINNER
    def winner(self):
        #row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1 #x win
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1 #loss x
            
        #col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1 #x win
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1 #loss x
            
        #diagonal
        diag1_sum = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag2_sum = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(abs(diag1_sum), abs(diag2_sum))
        if diag_sum == 3:
            self.isEnd = True
            if diag1_sum == 3 or diag2_sum == 3:
                return 1 #win x
            else:
                return -1 #loss x

        #draw, no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        
        #not end
        self.isEnd = False
        return None
    
    
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            return 1
        elif result == -1:
            return -1
        else:
            return 0.5

    #given the position = action = tupla that indicate the row and the colum where put the simbol 
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        #switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1

    

### Random player

In [148]:
class RandomPlayer:
    #costruttore
    def __init__(self, name):
        self.name = name

    #Making a random action/move
    def chooseAction(self, positions):
        idx = np.random.choice(len(positions))
        action = positions[idx]

        return action
    
    def reset(self):
        self.states = []


## Q-Learning Reinforcement Learning

### Q-Learning player 


In [149]:
class QPlayer:
    #learning rate. 0 < alpha < 1
    #discounted factor - cut-down constant. 0 < gamma < 1
    def __init__(self, name, learning_rate=0.1, discount_factor=0.5, exploration_prob=0.1):
        self.name = name 
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob
        #Init Q-table
        self.Q_table = []
   
    
    def chooseAction(self, positions, numberS: int):
        #sample a float from a uniform distribution over 0 and 1    
        if np.random.uniform(0, 1) <= self.exploration_prob: #if the sampled flaot is less than the exploration prob  
            #the player take a random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else: #the player chooses the action with the maximum q value
            action = max(self.Q_table[numberS,:])

        return action
           
    
    def update_q_table(self, numberS: int, numberA: int, reward, numberS1: int):    
        #Compute the max value of the next state
        max_q_next = max(self.Q_table[numberS1][numberA])
        
        #Update our Q-table using the Q-learning iteration
        self.Q_table[numberS][numberA] = (1 - self.learning_rate) * self.Q_table[numberS][numberA] + self.learning_rate * (reward + self.discount_factor * max_q_next)
  

In [150]:
availableNAction = set(range(1, 9))
availableNBoardState = set(range(1, 10))
mapBoard = {}
mapAction = {}
    
    
def getBoardStateNum(state: State):
    board = frozenset(state.get_board())
    if board not in mapBoard:
        mapBoard[board] = choice(list(availableNBoardState)) #choose random 
        numberBS =  mapBoard[board]   
        availableNBoardState.remove(numberBS)
        return numberBS
    else:
        return mapBoard[board]  
        

def getActionNum(action):
    action = frozenset(action)
    if action not in mapAction:
        mapAction[action] = choice(list(availableNAction)) #choose random 
        numberA =  mapAction[action]   
        availableNAction.remove(numberA)
        return numberA
    else:
        return mapAction[action]    
                       

In [151]:
#Runs the chosen action and returns  
def play(state, action):
    #take action and upate board state
    state.updateState(action)
    reward = state.giveReward()

    
    win = state.winner() #check winner
    if win is not None: #if the game is end
        return reward, state, state.isEnd
    else:
        #Player 2
        positions = state.availablePositions()
        p2_action = state.p2.chooseAction(positions)
                       
        state.updateState(p2_action)

        return reward, state, state.isEnd    


#### Training - RL to learn the real Q values

In [152]:
#Learning the state value function:
print("training...")

p1 = QPlayer("QPlayer")
p2 = RandomPlayer("Random")

for steps in tqdm(range(500)): 
    st = State(p1, p2)

    #initialize 
    st.reset()
        
    for i in range(100): #until the player life is over, iterate over episodes
        done = False
        current_state = copy.deepcopy(st)      
        numberS =  getBoardStateNum(current_state)

        positions = current_state.availablePositions()
        action  = p1.chooseAction(positions, numberS)
        numberA = getActionNum(action)
        
        #The environment runs the chosen action 
        reward, next_state, done = play(current_state, action)
        numberS1 =  getBoardStateNum(next_state)

        p1.update_q_table(numberS, numberA, reward, numberS1)   

        if done:
            break

print(p1.Q_table)

training...


  0%|          | 0/500 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not tuple

### Monte-Carlo - Reinforcement Learning 
Squillero's code: https://github.com/squillero/computational-intelligence/blob/master/2023-24/lab10-gx.ipynb


In [ ]:
# Definizione della tupla denominata "State" con i campi 'x' e 'o'
State = namedtuple('State', ['x', 'o'])

In [ ]:
#Restating the problem 
#A square array of numbers is called a magic square if the sums of the numbers in each row, each column, and both main diagonals are the same.
#I define a magic square with sum 15 since winning in tic tac toe is equivalent to taking three numbers from a 3x3 magic square whose sum is 15(in this example). 

MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8] #Magic Square

In [ ]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3): #row
        for c in range(3): #columns
            i = r * 3 + c
            if MAGIC[i] in pos.x: #checks if the current position is occupied by player x
                print('X', end='')
            elif MAGIC[i] in pos.o: #checks if the current position is occupied by player o
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()


#the win function returns True if there is a combination of three elements in elements whose sum is 15
def win(elements): 
    """Checks if elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))


def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1 #if player x has one winning combination
    elif win(pos.o):
        return -1 #if player o has a winning combination
    else:
        return 0

In [ ]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))

    while available:    
        x = choice(list(available)) #choose a random element
        state.x.add(x) #add an element x to the set x inside the state object
        trajectory.append(deepcopy(state))
        available.remove(x)
        if win(state.x) or not available: #The overall condition is true if the player x has won or if there are no more moves available.
            break

        o = choice(list(available)) #choose a random element
        state.o.add(o) #add an element o to the set x inside the state object
        trajectory.append(deepcopy(state))
        available.remove(o)
        if win(state.o):
            break

    return trajectory

#### Training

In [ ]:
#Player x

#Learning the state value function:
value_dictionary = defaultdict(float) #big dictionary state value during the training RL
hit_state = defaultdict(int) 
epsilon = 0.001 #learning rate

for steps in tqdm(range(50_000)):
    trajectory = random_game() #generate a random trajectory 
    final_reward = state_value(trajectory[-1]) #compute the reward given the last element of the trajectory list(the final state of the game)
    
    for state in trajectory: #For each state in the trajectory
        hashable_state = (frozenset(state.x), frozenset(state.o)) #use frozenset because they are hashable
        hit_state[hashable_state] += 1 #to count how many times a state was visited during training
        #updates the state rating model based on the difference between the final reward and the current rating:
        value_dictionary[hashable_state] = value_dictionary[hashable_state] + epsilon * (final_reward - value_dictionary[hashable_state])

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:07<00:00, 6355.11it/s]


In [ ]:
#Sorted in descending order based on the value e[1], the second element of the state-value tuple, si.e. the reward
value_dictionary_sorted = sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)

#give me the first ten items
count = 0
for key, value in value_dictionary_sorted:
    print(f"State: {key}, Reward: {value}")
    count += 1
    if count == 10:
        break

State: (frozenset({5}), frozenset()), Reward: 0.483826821814233
State: (frozenset({8}), frozenset()), Reward: 0.3469893262770049
State: (frozenset({6}), frozenset()), Reward: 0.3332456465906484
State: (frozenset({2}), frozenset()), Reward: 0.325877932450425
State: (frozenset({4}), frozenset()), Reward: 0.30119945258599656
State: (frozenset({5}), frozenset({3})), Reward: 0.2941284935819315
State: (frozenset({5}), frozenset({9})), Reward: 0.29305207043819664
State: (frozenset({5}), frozenset({1})), Reward: 0.2789096830671588
State: (frozenset({5}), frozenset({7})), Reward: 0.26579351800644657
State: (frozenset({2}), frozenset({9})), Reward: 0.25423810578347694


In [ ]:
len(hit_state)

5477